# File Downloading

In [4]:
import fcntl
import os
import requests




In [16]:
def load_model(model_path=None):
    # Load users model if specified and exists
    if model_path and os.path.isfile(model_path):
        clf = joblib.load(model_path)
    # Look for default model and if doesn't exist, download it
    else:
        default_model_path = 'aurora.jpg'
        if os.path.isfile(default_model_path):
            clf = joblib.load(default_model_path)
        else:
            url = 'https://eol.jsc.nasa.gov/DatabaseImages/EO/highres/ISS043/ISS043-E-86375.JPG'
            if not os.path.isfile(default_model_path):
                try:
                    os.makedirs(os.path.dirname(default_model_path))
                except OSError as exc: # Guard against rare condition
                    if exc.errno != errno.EEXIST:
                        raise
            with open(default_model_path, 'wb+') as f:
                try:
                    fcntl.flock(f, fcntl.LOCK_EX | fcntl.LOCK_NB)
                    if os.path.getsize(default_model_path) == 0:
                        r = requests.get(url, stream=True)
                        for chunk in r.iter_content(chunk_size=1024):
                            if chunk:  # filter out keep-alive new chunks
                                f.write(chunk)
                except BlockingIOError:
                    fcntl.flock(f, fcntl.LOCK_EX)
                finally:
                    fcntl.flock(x, fcntl.LOCK_UN)
                    
            
# ... make sure the directory exists ...
# with open(default_model_path, 'wb+') as f:
#     try:
#         fcntl.flock(f, fcntl.LOCK_EX | fcntl.LOCK_NB) # exclusive, nonblocking lock
#         ... check if the file is empty, if not, someone else has already finished downloading it ...
#         ... download and write the file ...
#     catch BlockingIOError:
#         # this happens if someone else is currently downloading it
#         fcntl.flock(f, fcntl.LOCK_EX)  # exclusive, *blocking* lock. will wait until the other lock is released
#     ...
#     finally:
#         fcntl.flock(x, fcntl.LOCK_UN) # probably not actually necessary because closing the filehandle will release the lock... I think.

In [6]:
os.path.isfile('files/happy.txt')

False

In [7]:
os.makedirs(os.path.dirname('files/happy.txt'))

In [17]:
load_model()

AttributeError: 'int' object has no attribute 'EEXIST'

In [14]:
OSError.errno

<member 'errno' of 'OSError' objects>